In [31]:
import os
import glob # folder operations
import re # use regular expressions
import logging 
# You perpahs need to restart the kernel in .ipynb to make the log module working normally

In [32]:
opcode_set={'bltlr+', 'bdz', 'stwcx.', 'stbux', 'vsububm', 'divw', 'stvx', 'stbu', 'cmpw', 'vsl', 
              'bcl', 'bne-', 'ble-', 'stwux', 'sraw', 'lvsl', 'stb', 'bdnz', 'mtfsf', 'ble+', 'srwi.',
              'cmplwi', 'subfe', 'beqlr+', 'bgt-', 'crset', 'subfc.', 'mulli', 'extsh', 'vncipher',
              'beqlr-', 'rlwinm.', 'sthu', 'andc.', 'mflr', 'fmuls', 'vsrab', 'vspltisw', 'bnel',
              'extrwi', 'cror', 'neg', 'addic', 'vpmsumd', 'addme', 'lhzu', 'mfspr', 'fabs', 'vaddubm',
              'blelr', 'addis', 'nop', 'vspltisb', 'lwsync', 'bne+', 'fctiwz', 'addic.', 'vaddudm',
              'vshasigmaw', 'andc', 'mtcrf', 'divwu', 'lis', 'srawi', 'lhzx', 'vsldoi', 'lwbrx', 
              'fcmpu', 'vspltb', 'mftb', 'fmadd', 'li', 'fmr', 'cmpwi', 'xxmrgld', 'mullw', 'add', 
              'stxvd2x', 'slw.', 'stbx', 'fsub', 'srawi.', 'lfdu', 'beq+', 'blt', 'not', 'vsel',
              'clrlslwi', 'fadd', 'addi', 'fneg', 'stw', 'beq-', 'cntlzw.', 'lvx', 'crnot', 'and',
              'extsb', 'fsubs', 'mcrf', 'lbz', 'frsp', 'lbzx', 'fadds', 'stwu', 'mffs', 'lwzux', 
              'isync', 'xor.', 'mulhwu', 'vnot', 'andis.', 'mtspr', 'subfe.', 'rlwimi.', 'cntlzw',
              'fdivs', 'sthx', 'insrwi', 'node', 'andi.', 'bltlr', 'fdiv', 'rlwinm', 'bgelr', 
              'extsb.', 'mr', 'subf.', 'extlwi', 'bctr', 'bltlr-', 'srw', 'twlt', 'lbzux', 
              'lvsr', 'bgt+', 'vcipher', 'rotrwi', 'bge+', 'sth', 'xxmrghd', 'vperm', 'srdi', 
              'maddhdu', 'addze', 'mfctr', 'add.', 'stvewx', 'xoris', 'vmr', 'vadduwm', 'ori', 
              'addc', 'cmplw', 'lxvw4x', 'srwi', 'lha', 'addze.', 'srw.', 'rlwimi', 'bge-', 'subfc',
              'beq', 'crclr', 'stfd', 'vmrgow', 'clrlwi', 'adde', 'fcfid', 'stfdu', 'not.', 'vand', 
              'bgtlr', 'bctrl', 'extrwi.', 'mtfsb0', 'vspltw', 'vslb', 'rotlw', 'oris', 'mr.', 
              'mtctr', 'stwx', 'eqv', 'maddld', 'beqlr', 'vsbox', 'lbzu', 'lwarx', 'xor', 'lfs',
              'vrlw', 'vsrb', 'vor', 'vmrgew', 'clrrwi.', 'bgt', 'lxvd2x', 'bnelr', 'blt+', 'dcbf',
              'vshasigmad', 'clrrwi', 'neg.', 'slwi.', 'subf', 'bge', 'subfze', 'lhax', 'xori', 
              'stxvw4x', 'bne', 'extsh.', 'lhau', 'slw', 'subfme', 'creqv', 'mtlr', 'vcipherlast',
              'nand', 'bl', 'nor', 'lhaux', 'vncipherlast', 'lwz', 'slwi', 'bnelr+', 'mulhw', 'or',
              'lwzx', 'blr', 'subfic', 'blelr+', 'mfcr', 'adde.', 'stvebx', 'lhz', 'lwzu',
              'orc', 'orc.', 'stwbrx', 'fmul', 'and.', 'mtfsb1', 'crxor', 'blt-', 'stfs',
              'ble', 'b', 'or.', 'fmsub', 'clrlwi.', 'mullw.', 'inslwi', 'lfd', 'rotlwi', 'vxor'}

register_set={'r0','r1','r2','r3','r4','r5','r6','r7','r8','r9','r10',
                  'r11','r12','r13','r14','r15','r16','r17','r18','r19','r20',
                  'r21','r22','r23','r24','r25','r26','r27','r28','r29','r30','r31',
                  'f0','f1','f2','f3','f4','f5','f6','f7','f8','f9','f10',
                  'f11','f12','f13','f14','f15','f16','f17','f18','f19','f20',
                  'f21','f22','f23','f24','f25','f26','f27','f28','f29','f30','f31',
                  'cr7'
                  }

dummy_set={'TAG','VALUE','HEX','FUNC','STR','DWORD','JPT','ASC','UNK','TBYTE','QWORD','XMMWORD','LONG',
               'DOUBLE','ARRAY','STRING','ARG','BYTE','SUB','WORD','LOCRET','DEF','OFF','FLT','DBL','LOC',
               'STRU','GPGRT','GPG','YMMWORD','BUF','FILE','CALLBACK','CB','CURLSH','BUFFER','SCWTCH','VAR','STR',
               'VAL','ALGN','SEG','SRC','INPUT','LOCAL','IMM','DUMMY','STD','ALLOC',
               'ha'}

In [33]:
def format_func_names(text, func_dict,lib):

    if lib=='binutils':
        func_set=func_dict['binutils']
    elif lib=='coreutils':
        func_set=func_dict['coreutils']
    elif lib=='curl':
        func_set=func_dict['curl']
    elif lib=='diffutils':
        func_set=func_dict['diffutils']
    elif lib=='findutils':
        func_set=func_dict['findutils']
    elif lib=='libgpg':
        func_set=func_dict['libgpg_error']    
    elif lib=='openssl':
        func_set=func_dict['openssl']
    elif lib=='zlib':
        func_set=func_dict['zlib']
    else: 
        func_set=set()
        print(f"Warning, undetected library: {lib}")                        

    #pattern=r'\b(' + '|'.join(func_set) + r')\b'
    pattern = r'\b(' + '|'.join(re.escape(func) for func in func_set) + r')\b(?=\s*\()'
    # Split the text into lines
    lines = text.split('\n')
    formatted_lines = []

    # Iterate over each line
    for line in lines:
        # Split the line into the first word and the rest of the line
        parts = line.split(' ', 1)
        first_word = parts[0]
        rest_of_line = parts[1] if len(parts) > 1 else ''
        original_line=rest_of_line
        # Replace occurrences of any function name in the rest of the line with '<FUNC>'
        modified_rest_of_line = re.sub(r'_\b(' + '|'.join(func_set) + r')\b', ' <FUNC>', rest_of_line)
        modified_rest_of_line = re.sub(pattern, ' <FUNC>', rest_of_line)
        
        #modified_rest_of_line = re.sub(underscore_pattern, ' <FUNC>', modified_rest_of_line)
        #print(pattern)
        # Concatenate the first word and the modified rest of the line
        #if original_line != modified_rest_of_line:
            #logging.info(f"Formatting function: {original_line} -> {modified_rest_of_line}")

        formatted_line = first_word + (' ' + modified_rest_of_line if modified_rest_of_line else '')
        formatted_lines.append(formatted_line)

    # Join the formatted lines back into text
    formatted_text = '\n'.join(formatted_lines)
    return formatted_text

In [34]:
def format_call_func(text):
    """
    Converts lines starting with 'call' to '<FUNC_CALL>'.

    Parameters:
    - text (str): The input text in which to convert the lines.

    Return
    - str: The text with 'call' lines converted.The exact function is replaced with '<FUNC>'
    """
    # Split the text into lines
    lines = text.split('\n')
    # Convert lines starting with 'call'
    converted_lines = ['bl <FUNC>' if line.lstrip().startswith('bl') else line for line in lines]
    # Join the lines back into a single string
    return '\n'.join(converted_lines)

In [35]:
def format_suffix_vars(text):
    lines=text.split('\n')
    #opcode_pattern='|'.join(re.escape(opcode) for opcode in opcode_set)

    aLarge_pattern=re.compile(r'\s(a[A-Z][a-z0-9A-Z_\.]+)\_*\d*')# match anything except for a lower or upper case letter
    dword_pattern=re.compile(r'\s(dword_[0-9a-zA-Z_\.]+)')
    jpt_pattern=re.compile(r'\s(jpt_[0-9a-zA-Z_\.]+)')
    asc_pattern=re.compile(r'\s(asc_[0-9a-zA-Z_\.]+)')
    unk_pattern=re.compile(r'\s(unk_[0-9a-zA-Z_\.]+)')
    tbyte_pattern=re.compile(r'\s(tbyte_[0-9a-zA-Z_\.]+)')
    qword_pattern=re.compile(r'\s(qword_[0-9a-zA-Z_\.]+)')
    xmmword_pattern=re.compile(r'\s(xmmword_[0-9a-zA-Z_\.]+)')
    long_pattern=re.compile(r'\s(long_[0-9a-zA-Z_\.]+)')
    double_pattern=re.compile(r'\s(double_[0-9a-zA-Z_\.]+)')
    array_pattern=re.compile(r'\s(array_[0-9a-zA-Z_\.]+)')
    string_pattern=re.compile(r'\s(string_[0-9a-zA-Z_\.]+)')
    arg_pattern=re.compile(r'\s(arg_[0-9a-zA-Z_\.]+)')
    byte_pattern=re.compile(r'\s(byte_[0-9a-zA-Z_\.]+)')
    sub_pattern=re.compile(r'\s(sub_[0-9a-zA-Z_\.]+)')
    word_pattern=re.compile(r'\s(word_[0-9a-zA-Z_\.]+)')
    locret_pattern=re.compile(r'\s(locret_[0-9a-zA-Z_\.]+)')
    def_pattern=re.compile(r'\s(def_[0-9a-zA-Z_\.]+)')
    off_pattern=re.compile(r'\s(off_[0-9a-zA-Z_\.]+)')
    flt_pattern=re.compile(r'\s(flt_[0-9a-zA-Z_\.]+)')
    dbl_pattern=re.compile(r'\s(dbl_[0-9a-zA-Z_\.]+)')
    loc_pattern=re.compile(r'\s(loc_[0-9a-zA-Z_\.]+)')
    stru_pattern=re.compile(r'\s(stru_[0-9a-zA-Z_\.]+)')
    gpgrt_pattern=re.compile(r'\s(gpgrt_[0-9a-zA-Z_\.]+)')
    gpg_pattern=re.compile(r'\s(gpg_[0-9a-zA-Z_\.]+)')
    ymmword_pattern=re.compile(r'\s(ymmword_[0-9a-zA-Z_\.]+)')
    buf_pattern=re.compile(r'\s(buf_[0-9a-zA-Z_\.]+)')
    FILE_pattern=re.compile(r'\s(FILE_[0-9a-zA-Z_\.]+)')
    callback_pattern=re.compile(r'\s(callback_[0-9a-zA-Z_\.]+)')
    cb_pattern=re.compile(r'\s(cb_[0-9a-zA-Z_\.]+)')
    CURLSH_pattern=re.compile(r'\s(CURLSH_[0-9a-zA-Z_\.]+)')
    buffer_pattern=re.compile(r'\s(buffer_[0-9a-zA-Z_\.]+)')
    CSWTCH_pattern=re.compile(r'\s(CSWTCH_[0-9a-zA-Z_\.]+)')
    var_pattern=re.compile(r'\s(var_[0-9a-zA-Z_\.]+)')
    str_pattern=re.compile(r'\s(str_[0-9a-zA-Z_\.]+)')
    val_pattern=re.compile(r'\s(val_[0-9a-zA-Z_\.]+)')
    pretty_function_pattern=re.compile(r'\s(__PRETTY_FUNCTION___\d+)')
    algn_pattern=re.compile(r'\s(algn_[0-9a-zA-Z_\.]+)')
    seg_pattern=re.compile(r'\s(seg_[0-9a-zA-Z_\.]+)')
    src_pattern=re.compile(r'\s(src_[0-9a-zA-Z_\.]+)')
    input_pattern=re.compile(r'\s(input_[0-9a-zA-Z_\.]+)')
    local_pattern=re.compile(r'\s(local_[0-9a-zA-Z_\.]+)')
    imm_pattern=re.compile(r'([\s#\.])imm[0-9a-zA-Z_\.]+')
    dummy_pattern=re.compile(r'([\s#\.])dummy[0-9a-zA-Z__]*')
    std_pattern=re.compile(r'([\s#\.])std_[0-9a-zA-Z_\.]+')
    alloc_pattern=re.compile(r'\s(alloc_[0-9a-zA-Z_]+)')

    formatted_lines=[]
    for line in lines:
        original_line=line

        line=aLarge_pattern.sub(' <STR>',line)
        line=dword_pattern.sub(' <DWORD>',line)
        line=jpt_pattern.sub(' <JPT>',line)
        line=asc_pattern.sub(' <ASC>',line)
        line=unk_pattern.sub(' <UNK>',line)
        line=tbyte_pattern.sub(' <TBYTE>',line)
        line=qword_pattern.sub(' <QWORD>',line)
        line=xmmword_pattern.sub(' <XMMWORD>',line)
        line=long_pattern.sub(' <LONG>',line)
        line=double_pattern.sub(' <DOUBLE>',line)
        line=array_pattern.sub(' <ARRAY>',line)
        line=string_pattern.sub(' <STRING>',line)
        line=arg_pattern.sub(' <ARG>',line)
        line=byte_pattern.sub(' <BYTE>',line)
        line=sub_pattern.sub(' <SUB>',line)
        line=word_pattern.sub(' <WORD>',line)
        line=locret_pattern.sub(' <LOCRET>',line)
        line=def_pattern.sub(' <DEF>',line)
        line=off_pattern.sub(' <OFF>',line)
        line=flt_pattern.sub(' <FLT>',line)
        line=dbl_pattern.sub(' <DBL>',line)
        line=loc_pattern.sub(' <LOC>',line)
        line=stru_pattern.sub(' <STRU>',line)
        line=gpgrt_pattern.sub(' <GPGRT>',line)
        line=gpg_pattern.sub(' <GPG>',line)
        line=ymmword_pattern.sub(' <YMMWORD>',line)
        line=buf_pattern.sub(' <BUF>',line)
        line=FILE_pattern.sub(' <FILE>',line)
        line=callback_pattern.sub(' <CALLBACK>',line)
        line=cb_pattern.sub(' <CB>',line)
        line=CURLSH_pattern.sub(' <CURLSH>',line)
        line=buffer_pattern.sub(' <BUFFER>',line)
        line=CSWTCH_pattern.sub(' <SCWTCH>',line)
        line=var_pattern.sub(' <VAR>',line)
        line=str_pattern.sub(' <STR>',line)
        line=val_pattern.sub(' <VAL>',line)
        line=pretty_function_pattern.sub(' <FUNC>',line)
        line=algn_pattern.sub(" <ALGN>",line)
        line=seg_pattern.sub(" <SEG>",line)
        line=src_pattern.sub(" <SRC>",line)
        line=input_pattern.sub(" <INPUT>",line)
        line=local_pattern.sub(" <LOCAL>",line)
        line=imm_pattern.sub(" <IMM>",line)
        line=dummy_pattern.sub(" <DUMMY>",line)
        line=std_pattern.sub(" <STD>",line)
        line=alloc_pattern.sub(" <ALLOC>",line)

        if original_line != line:
            logging.info(f"Formatting suffix: {original_line} -> {line}")
        
        formatted_lines.append(line)
    formatted_text='\n'.join(formatted_lines)    
    return formatted_text 

In [36]:
def format_values(text):
    """
    It identifies the hex value with '0x' prefix. If the digits exceeds 4, replace it
    with '0x_VALUE', otherwise keep it. 
    """
    hex_pattern1 = re.compile(r'0x[a-fA-F0-9]+')# regular expression for hex value
                                                  # more than 4 digits
    #hex_pattern2 = re.compile(r'\s[a-fA-F0-9]+h\s')   
    decimal_pattern=re.compile(r'\s\d+')                                           
    lines = text.split('\n') # Split the text into lines
    converted_lines = []
    for line in lines:
        original_line=line
        line = hex_pattern1.sub(' <HEX>', line)  # Apply the first pattern
        #line = re.sub(hex_pattern2,' <HEX> ',line)  # Apply the second pattern
        line = re.sub(decimal_pattern,' <VALUE>',line)
        # if original_line != line:
        #     logging.info(f"Formatting VALUE: {original_line} -> {line}")
        converted_lines.append(line)
    #it replaces all the patterns with 'hex_VALUE' in the line
    # Return the processed text
    return '\n'.join(converted_lines)

In [37]:
def format_TAG(text,dummy_set,opcode_set,register_set):
        
    #dash_var_pattern=re.compile(rf'(?!{opcode_pattern})[0-9a-zA-Z_\.]+([_|\.]+[0-9a-zA-Z]+[_]*)+')
    #dash_var_pattern=re.compile(r'(?<![0-9a-zA-Z_\.])[0-9a-zA-Z_\.]+([_|\.]+[0-9a-zA-Z]+[_]*)+(?=\s)')
    
    tag_pattern1 = re.compile(r'\b([a-zA-Z0-9\.]+_[a-zA-Z0-9\.]+)\b')
    tag_pattern2 = re.compile(r"\b([a-zA-Z0-9_\.]{8,})\b")
    
    tag_pattern = re.compile(r'(?<![a-zA-Z0-9_\.])(?<!\.)[a-zA-Z0-9_\.]{2,}(?=\s)')

    lines = text.split('\n')
    formatted_lines = []

    for line in lines:
        words2 = tag_pattern2.findall(line)
        #This step guarantees the longer match is replaced first
        words2.sort(key=len, reverse=True)
        #logging.info(f"The matched words include: {words}")
        formatted_line = line
        for word in words2:
            if word not in opcode_set|register_set|dummy_set:  
                #logging.info(f"The word <{word}> will be replaced.")
                formatted_line = re.sub(rf'\b{word}\b', "<TAG>", formatted_line)
        
        words1 = tag_pattern.findall(formatted_line)
        words1.sort(key=len, reverse=True)
        for word in words1:
            if word not in opcode_set|register_set|dummy_set:  
                #logging.info(f"The word <{word}> will be replaced.")
                formatted_line = re.sub(rf'\b{word}\b', "<TAG>", formatted_line)      

        if formatted_line!=line:
            logging.info(f"Formatting TAG: {line} -> {formatted_line}")

        formatted_lines.append(formatted_line)

        # Join the formatted lines back into text
    formatted_text = '\n'.join(formatted_lines)
    return formatted_text

In [38]:
def replace_words(text, opcode_set,register_set,dummy_set):
    #This function replaces the words longer than 3 letters with <TAG> except
    #for the opcode
    lines = text.split('\n')
    formatted_lines = []
    
    
    words_pattern = re.compile(r'[a-zA-Z0-9]{5,}(?=\s)')  # Regular expression pattern to match lowercase words with more than 3 letters
    for line in lines:
        words = words_pattern.findall(line)
        #This step guarantees the longer match is replaced first
        words.sort(key=len, reverse=True)
        formatted_line = line
        for word in words:
            if word not in opcode_set|register_set|dummy_set:
                logging.info(f"The word <{word}> is not in any sets and will be replaced ")
                formatted_line = re.sub(rf'\b{word}\b', "<TAG>", formatted_line)
                #formatted_line = formatted_line.replace(word, "<TAG>")
        if formatted_line!=line:
            logging.info(f"Formatting WORD: {line} -> {formatted_line}")

        formatted_lines.append(formatted_line)
   
    return '\n'.join(formatted_lines)   

In [39]:
def remove_comments(text):
    """
    Removes comments that appear after "//" and ';' in each line.

    Parameters:
    - text (str): The input text from which to remove the comments.

    Returns:
    - str: The text with comments removed.
    """
    # Split the text into lines
    lines = text.split('\n')
    # Remove comments from each line
    cleaned_lines = [line.split('//')[0].rstrip() for line in lines]
    cleaned_lines2=[line.split(';')[0].rstrip() for line in cleaned_lines]
    cleaned_lines3=[line.split('"')[0].rstrip() for line in cleaned_lines2]# the content after '"' should also be deleted. 
    cleaned_lines4 = [line.split('node')[0] + 'node' if 'node' in line else line for line in cleaned_lines3]
    cleaned_lines5=[line.split('#')[0].rstrip() for line in cleaned_lines4]
    # Join the lines back into a single string
    return '\n'.join(cleaned_lines5)

In [40]:
def remove_extra_lines(text):
    """
    Removes lines that contain 'colorentry' and the numbers that follow.
    We also remove the lines that start with 'edge:'. 
    We don't need those information. 

    Parameters:
    - text (str): The input text from which to remove the lines.

    Returns:
    - str: The text with 'colorentry' lines removed.
    """
    # Split the text into lines
    lines = text.split('\n')
    # Remove lines containing 'colorentry'
    lines=lines[10:]
    cleaned_lines = [line for line in lines if not line.startswith('colorentry')
                     |line.startswith('edge')|line.startswith('}')|line.startswith('IMPORT')|line.startswith('.extern')]
    # Join the lines back into a single string
    return '\n'.join(cleaned_lines)

In [41]:
def tokenize(text):
    lines=text.split('\n')
    for i,line in enumerate(lines):
        token_pattern=re.compile(r'([\[\]\+\:\-\(\)\*\{\}])')
        prefix_pattern=re.compile(r'(@)')
        replace_pattern=re.compile(r'(\,)')
        sign_matches=token_pattern.findall(line) 
        line = replace_pattern.sub(r' ',line)
        line = prefix_pattern.sub(r' \1', line)
        line = token_pattern.sub(r' \1 ', line)
        line = re.sub(r'\s+', ' ', line)
        lines[i] = line
    return '\n'.join(lines)    

In [42]:
def remove_empty_lines(text):
    '''
    This function is used to delete empty lines in the formatted gdl file. 
    It also removed the fode information
    '''
    lines=text.split('\n')
    new_lines=[line for line in lines if line.strip()!='']
    cleaned_text='\n'.join(new_lines)
    return cleaned_text  

In [43]:
def formulate_sentence(text):
    #this function formulates all the instructions within one node to a whole sentence. 
    #it also deletes all the commas in the text 
    lines=text.split('\n')
    output=''
    current_line=''
    for line in lines:
        if line.strip()=='node':
            if current_line:
                output+=current_line.strip().replace(",", "")+'\n'
                current_line=""
        else:
            current_line+=line.strip().replace(",", "")+' <E> '
    return output

In [44]:
def process_file(gdl_file_path, output_path, func_dict,opcode_set,register_set,dummy_set):
    """
    Reads a GDL file, applies cleaning functions, and writes the output to a new file.

    Parameters:
    - gdl_file_path (str): The path to the GDL file to be processed.
    """
    # Extract the directory and the base name of the gdl file
    file_dir, base_name = os.path.split(gdl_file_path)
    #print(f"the file directory is {file_dir}")
    #output_file_dir=file_dir.split('/')[-1]

    #The following command is to remove the version information so the comparison is easier
    new_output_file_dir=(file_dir.split('-')[0]+'-'+file_dir.split('-')[-1]).split('/')[-1]
    #print(f"the new output file directory is {new_output_file_dir}")
    #print(f"the output file directory is {output_file_dir}")
    # Remove the '.gdl' extension and add '_postprocessed'
    output_base_name = base_name[:-4]
    #output_file_path = os.path.join(file_dir, output_base_name)
    #print(f"the output file path is {output_base_name}")  
    output_name=new_output_file_dir+'-'+output_base_name
    #print(f"the output name is {output_name}")
    output_file_path=os.path.join(output_path,output_name)

#     if output_name.split('-')[0] not in{'binutils','openssl','findutils','libgpg'}:
#          logging.info(f"{gdl_file_path} not in the training dataset, skipping. ")
#          return

    if os.path.exists(output_file_path):
        logging.info(f"Skipping file: {output_file_path} (output file already exists)")
        return

    # Read the input GDL file
    with open(gdl_file_path, 'r') as file:
         text = file.read()

    text = remove_extra_lines(text)
    text = remove_comments(text)
    text = remove_empty_lines(text)
    #text = format_call_func(text)

    #text=format_call_func(text)
    text = tokenize(text)
    # the last parameter corresponds to the lib: findutils, libgpg, openssl etc
    #text = format_func_names(text,func_dict,new_output_file_dir.split('-')[0]) 
    text = format_suffix_vars(text)
    
    text = format_values(text) 
    text = format_TAG(text,opcode_set,register_set,dummy_set)
    #text = replace_words(text,opcode_set,register_set,dummy_set)
    text = formulate_sentence(text)
    
    #Write the cleaned text to the output file
    with open(output_file_path, 'w') as file:
         file.write(text)

In [45]:
def main():
    ISA='ppc32'
    log_path = f"../original_dataset/gdl_pb_formatted/log/{ISA}"
    os.makedirs(log_path, exist_ok=True)
    log_file = os.path.join(log_path, f"{ISA}_formatting.log")
    logging.basicConfig(filename=log_file, level=logging.INFO, format='%(message)s')
    
    # Use glob to find all '.gdl' files recursively in the current folder and sub-folders
    input_path = glob.glob(f'../original_dataset/gdl_pb/{ISA}_malware/*.gdl', recursive=True)
    #input_path = glob.glob(f'../original_dataset/gdl_pb/{ISA}/**/**/*.gdl', recursive=True)
    #print(input_path)
    output_path=f"../original_dataset/gdl_pb_formatted/{ISA}_malware"
    #output_path=f"../Malware_detection_files/tensors/{ISA}_translation/{ISA}_malware"
    os.makedirs(output_path, exist_ok=True)

    #opcode_set_path=f'../original_dataset/opcode_operand/{ISA}/{ISA}_opcode_counts.txt'
    #register_set_path=f'../original_dataset/opcode_operand/{ISA}/register_counts.txt'

    zlib_func_set_path='../original_dataset/func_set/ppc32_func_set/ppc32_zlib-1.2.11-bin_func.txt'
    binutils_func_set_path='../original_dataset/func_set/ppc32_func_set/ppc32_binutils-2.34-bin_func.txt'
    coreutils_func_set_path='../original_dataset/func_set/ppc32_func_set/ppc32_coreutils-9.0-bin_func.txt'
    curl_func_set_path='../original_dataset/func_set/ppc32_func_set/ppc32_curl-7.87.0-bin_func.txt'
    diffutils_func_set_path='../original_dataset/func_set/ppc32_func_set/ppc32_diffutils-3.7-bin_func.txt'
    findutils_func_set_path='../original_dataset/func_set/ppc32_func_set/ppc32_findutils-4.8.0-bin_func.txt'
    libgpg_error_func_set_path='../original_dataset/func_set/ppc32_func_set/ppc32_libgpg-error-1.45-bin_func.txt'
    openssl_func_set_path='../original_dataset/func_set/ppc32_func_set/ppc32_openssl-1.1.1p-bin_func.txt'

    zlib_func_set=set()
    binutils_func_set=set()
    coreutils_func_set=set()
    curl_func_set=set()
    diffutils_func_set=set()
    findutils_func_set=set()
    libgpg_error_func_set=set()
    openssl_func_set=set()

    with open(zlib_func_set_path,'r') as file:
        for line in file:
            zlib_func_set.add(line.strip())  

    with open(binutils_func_set_path,'r') as file:
        for line in file:
            binutils_func_set.add(line.strip())   

    with open(coreutils_func_set_path,'r') as file:
        for line in file:
            coreutils_func_set.add(line.strip())  

    with open(curl_func_set_path,'r') as file:
        for line in file:
            curl_func_set.add(line.strip())  

    with open(diffutils_func_set_path,'r') as file:
        for line in file:
            diffutils_func_set.add(line.strip())  

    with open(findutils_func_set_path,'r') as file:
        for line in file:
            findutils_func_set.add(line.strip())  

    with open(libgpg_error_func_set_path,'r') as file:
        for line in file:
            libgpg_error_func_set.add(line.strip())  

    with open(openssl_func_set_path,'r') as file:
        for line in file:
            openssl_func_set.add(line.strip())        

    func_dict={'zlib':zlib_func_set, 'binutils':binutils_func_set,'openssl':openssl_func_set, 
               'findutils':findutils_func_set,'diffutils':diffutils_func_set,'libgpg_error':libgpg_error_func_set,
               'curl':curl_func_set,'coreutils':coreutils_func_set}

    # Process each file
    for gdl_file_path in input_path:
        #if gdl_file_path.split('-')[0] in{'binutils','openssl','findutils','libgpg'}:
        process_file(gdl_file_path, output_path,func_dict,opcode_set,register_set,dummy_set)
if __name__ == '__main__':
    main()